In [ ]:
import re
import numpy as np
import pandas as pd
from textwrap import dedent
from tqdm.auto import tqdm

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"

In [ ]:
graded_zs_df = pd.read_csv("results/trustworthy_zs_gpt-4.1-mini_graded.csv")
graded_zst_df = pd.read_csv("results/trustworthy_zst_gpt-4.1-mini_graded.csv")
graded_react_df = pd.read_csv("results/trustworthy_react_gpt-4.1-mini_graded.csv")
graded_planact_df = pd.read_csv("results/trustworthy_planact_gpt-4.1-mini_graded.csv")
graded_planreact_df = pd.read_csv("results/trustworthy_planreact_gpt-4.1-mini_graded.csv")

In [ ]:
graded_zs_df.head()

,task_id,step,question,answer,prompt,raw_action,finish_response,trust_score,f1,action_name,action_params,llm_grade,self_eval_score
0,d71ab288-8d4e-4e8b-9c3c-2637c408441f,0,Who was once considered the best kick boxer in...,Badr Hari,You are an intelligent Wikipedia agent. You sh...,"Wikipedia_Search[{""query"": ""best kickboxer con...",NaN,0.940568,NaN,Wikipedia_Search,{'query': 'best kickboxer controversies unspor...,NaN,NaN
1,d71ab288-8d4e-4e8b-9c3c-2637c408441f,1,Who was once considered the best kick boxer in...,Badr Hari,You are an intelligent Wikipedia agent. You sh...,"Wikipedia_Search[{""query"": ""kickboxer controve...",NaN,0.934348,NaN,Wikipedia_Search,{'query': 'kickboxer controversies unsportsman...,NaN,NaN
2,d71ab288-8d4e-4e8b-9c3c-2637c408441f,2,Who was once considered the best kick boxer in...,Badr Hari,You are an intelligent Wikipedia agent. You sh...,"Wikipedia_Search[{""query"": ""kickboxer best in ...",NaN,0.892140,NaN,Wikipedia_Search,{'query': 'kickboxer best in the world controv...,NaN,NaN
3,d71ab288-8d4e-4e8b-9c3c-2637c408441f,3,Who was once considered the best kick boxer in...,Badr Hari,You are an intelligent Wikipedia agent. You sh...,"Wikipedia_Search[{""query"": ""best kickboxer con...",NaN,0.902625,NaN,Wikipedia_Search,{'query': 'best kickboxer controversies violen...,NaN,NaN
4,d71ab288-8d4e-4e8b-9c3c-2637c408441f,4,Who was once considered the best kick boxer in...,Badr Hari,You are an intelligent Wikipedia agent. You sh...,"Wikipedia_Search[{""query"": ""best kickboxer con...",NaN,0.926903,NaN,Wikipedia_Search,{'query': 'best kickboxer controversies violen...,NaN,NaN


In [ ]:
def construct_confidence_prompt(full_prompt_and_actions: str, finish_action: str) -> str:
    template = f"""
Interaction:
{full_prompt_and_actions}{finish_action}

Instruction:
Evaluate whether the Finish action's "response" correctly and completely answers the task described in the Interaction.

Output (strictly):
Explanation: [one- or two-sentence rationale]
Correct: [Yes or No]"""
    return dedent(template)

def parse_self_eval_score(text: str) -> float:
    t = text.lower()
    if "correct: yes" in t:
        return 1.0
    if "correct: no" in t:
        return 0.0
    return np.nan

def get_self_eval_score(llm, full_prompt_and_actions: str, finish_action: str) -> float:
    """Run the LLM and parse out a 1.0/0.0 score."""
    prompt = construct_confidence_prompt(full_prompt_and_actions, finish_action)
    resp = llm(prompt)
    return parse_self_eval_score(resp)

In [ ]:
tqdm.pandas()

def add_self_eval_scores(
    df: pd.DataFrame,
    llm,
) -> pd.DataFrame:
    """
    Returns a new DataFrame with a `self_eval_score` column.

    - Uses `prompt` as the full interaction text for every row.
    - Runs `get_self_eval_score` on rows where `action_name` == 'Finish'.
    - Leaves NaN elsewhere.
    """
    def _compute(row):
        if row["action_name"] == "Finish":
            return get_self_eval_score(
                llm,
                row["prompt"],
                row["raw_action"]
            )
        return np.nan

    df = df.copy()
    # this will show a tqdm bar in the notebook
    df["self_eval_score"] = df.progress_apply(_compute, axis=1)
    return df

In [ ]:
import openai

def llm(prompt: str) -> str:
    reply = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[{"role":"system", "content": "You are a helpful assistant."}, 
                    {"role":"user","content":prompt}],
        temperature=0.0,
    )
    return reply.choices[0].message.content

Here’s an example of a self-evaluation prompt.

In [ ]:
finish_row = graded_zs_df[graded_zs_df['action_name'] == 'Finish'].iloc[0]

full_prompt       = finish_row['prompt']
finish_action_raw = finish_row['raw_action']

print(construct_confidence_prompt(full_prompt, finish_action_raw))


Interaction:
You are an intelligent Wikipedia agent. You should follow your [Role], [Action_Doc] to take actions. You should decompose your task into executable actions. Your generation should follow the example format. Finish the task if you find the answer. And your answer should be simple and straighforward. You can only take one action per generation. DO NOT repeat your actions.
[Role]
Answer questions by searching Wikipedia content.
[End of Role]
[Constraint]
Generation should be simple and clear.
[End of Constraint]
[Action_Doc]
[{'name': 'Wikipedia_Search', 'description': 'Using this API to search Wiki content.', 'parameters': {'query': 'the search string. be simple.'}}, {'name': 'Finish', 'description': 'Complete the task with a response.', 'parameters': {'response': 'this is the finish action response. Respond towards the task instruction.'}}]
[End of Action_Doc]
Using the following action format example to generate well formatted actions.
[ActionFormatExample]
Action: Wikipe

In [ ]:
graded_planreact_df = add_self_eval_scores(graded_planreact_df, llm)
graded_planreact_df.to_csv("results/trustworthy_planreact_gpt-4.1-mini_graded.csv", index=False)

  0%|          | 0/1941 [00:00<?, ?it/s]